# Oogeso - test case 2
Simple case for illustrating/testing the Oogeso model

In [ ]:
%load_ext autoreload
%autoreload 2
from oogeso import multicarrier
from oogeso import plots
import matplotlib.pyplot as plt
import IPython
import pyomo.environ as pyo
import logging
import pandas as pd
plots.plotter='plotly'
import plotly.express as px

In [ ]:
data0 = multicarrier.read_data_from_yaml('test case2.yaml')
profiles = multicarrier.read_profiles_from_xlsx('test case2 profiles.xlsx')

# MODIFY input data
for dev in ['GT1','GT2']:
    data0['paramDevice'][dev]['isOn_init'] = 1
#data['paramParameters']['elReserveMargin'] = 0

data = multicarrier.create_initdata(data0)
mc = multicarrier.Multicarrier(loglevel="INFO")
mc.createModelInstance(data,profiles)

In [ ]:
IPython.display.Image(plots.plotNetwork(mc,timestep=None).create_png())

In [ ]:
status = mc.solveMany(solver="cbc",timerange=[0,90],write_yaml=False)

In [ ]:
#mc.instance.constrDevice_startup_delay._data['GT2',0].pprint()
#mc.instance.constrDevice_startup_delay.clear()
#mc.instance.constrDevice_startup_delay.construct()
#mc.instance.constrDevice_startup_delay._data
#mc.instance.constrDevice_startup_delay.reconstruct()
mc.instance.constrTerminalEnergyBalance['el','node1','out',0].pprint()
mc.instance.constrTerminalEnergyBalance['el','node1','in',0].pprint()
#mc.instance.constrDevice_gasturbine.pprint()
mc.instance.constrDevice_ramprate['GT2',1].pprint()

#mc.instance.pprint()

In [ ]:
gt='GT2'
print("GT is on initially ={}".format(pyo.value(mc.instance.paramDeviceIsOnInitially[gt])))
print("GT timesteps in startup preparation = {}".format(pyo.value(mc.instance.paramDevicePrepTimestepsInitially[gt])))
for t in range(4):
    cstr=mc.instance.constrDevice_startup_shutdown[gt,t]
    print("t={}: {} = {}".format(t,cstr.body,cstr.upper))
for t in range(4):
    cstr=mc.instance.constrDevice_startup_delay[gt,t]
    print("t={}: {} = {}".format(t,cstr.body,cstr.upper))
for t in range(4):
    cstr=mc.instance.constrDevicePmax[gt,t]
    print("t={}: {} = {}".format(t,cstr.body,cstr.upper))

In [ ]:
dotG=plots.plotNetwork(mc,timestep=15,filename=None)
#IPython.display.SVG(dotG.create_svg()) #original size, vector graphics
IPython.display.Image(dotG.create_png()) #scales automatically to page width

In [ ]:
plots.plot_SumPowerMix(mc,carrier="el").show()

In [ ]:
# Problem = isOn = isPrep, isStarting=0
gt='GT2'
dfStart=pd.concat([
    mc._dfDeviceStarting[gt],
    mc._dfDeviceIsPrep[gt],
    mc._dfDeviceIsOn[gt],
    mc._dfDeviceStopping[gt]],axis=1)
dfStart.columns=['starting','prep','on','stopping']
#dfStart[12:25]

In [ ]:
#mc.instance.varDeviceIsPrep.pprint()

In [ ]:
plots.plot_deviceprofile(mc,devs=['GT1','GT2','GT3'],includeOnOff=True,includePrep=True)

In [ ]:
#plots.plot_CO2rate(mc)
plots.plot_CO2rate_per_dev(mc)

In [ ]:
plots.plotReserve(mc)

In [ ]:
plots.plotElBackup(mc,showMargin=True)

In [ ]:
dfplot=mc._dfElReserve
margin=mc.instance.paramParameters['elReserveMargin']
x = [dfplot.index[0],dfplot.index[-1]]
fig=px.line(dfplot,title="Reserve")
fig.add_scatter(x=x,y=[margin,margin],line=dict(dash='dot'),name="margin",mode="lines")
fig.update_xaxes(title_text="Timestep")
fig.update_yaxes(title_text="Reserve power (MW)",dtick = 5)
fig.update_layout(autosize=False,width=800,height=300,margin=dict(l=0,r=0,t=30,b=0))

In [ ]:
if 'battery' in mc.instance.setDevice:
    fig=plots.plot_devicePowerEnergy(mc,'battery',
                                 filename=None)
    fig.update_layout(autosize=False,width=800,height=300,margin=dict(l=0,r=0,t=30,b=0)).show()
    print("Last optimisation scheduling:")
    multicarrier.Plots.plotDevicePowerLastOptimisation1(mc,device='battery',
                filename=None)